# Fresh take

## 1. Initalize sites

In [1]:
%load_ext autoreload
import os
import os
import shutil
import dbPipeline
import time
from parseFiles.helperFunctions.loadDict import loadDict

In [2]:
data_dump = r'U:\data-dump'

In [ ]:
%autoreload 2
projectPath = os.path.join(os.getcwd(),'test')
try:
    shutil.rmtree(projectPath)
except:
    pass
db = dbPipeline.database(projectPath=projectPath,verbose=True,enableParallel=True)
Sites = loadDict(r'example_data\projectInventory_Template.yml')
# print(Sites)

db.projectInventory(newSites=Sites)

# template = loadDict(r'example_data\source_Template.yml')

# print(template)

# siteID,measurementID = 'SCL','Met'

# dpath = r'SCL\2024\20240912'
# template[siteID][measurementID]['rootPath'] = os.path.join(data_dump,dpath)
# print(template[siteID][measurementID]['rootPath'])
# db.rawFileSearch(siteID,measurementID,template[siteID][measurementID])

Creating template project inventory
None
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\siteID\siteID_metadata.yml
c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\siteID\measurementID\sourceFiles.yml

does not exist, creating new file using template
line 25:
{'w*ldcard': {'matchPattern': 'w*ldcard', 'rootPath': None, 'fileList': {}, 'parserSettings': {}}}

Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\siteID\measurementID\sourceFiles.yml
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\siteID\siteID_metadata.yml
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\siteID\measurementID\sourceFiles.yml


In [14]:
%autoreload 2

df = dbPipeline.databaseFolder(path=r'C:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Met',Years=[2024])
df.dataOut.columns

Index(['POSIX_timestamp'], dtype='object')

In [15]:
df.dataOut.loc[df.dataOut['SlrW_Avg']<df.dataOut['SlrW_Avg'].quantile(.99),'SlrW_Avg'].plot()#.to_csv(r'C:\Users\jskeeter\OneDrive - NRCan RNCan\Documents\Presentations\CANCH4\Data\SCL_met.csv')

KeyError: 'SlrW_Avg'

In [ ]:
from parseFiles.helperFunctions.loadDict import loadDict
# importlib.reload(dbPipeline)

# db = dbPipeline.database(projectPath=projectPath,verbose=False,enableParallel=False)
template = loadDict(r'example_data\source_Template.yml')
# # sourcePath = r'U:\EC_Backup\SCL_2024\20240724'
sourcePath = r'example_data'
print(template)
for siteID in template:
    for measurementID in template[siteID]:
        db.rawFileSearch(siteID,measurementID,template[siteID][measurementID])